### Example notebook to analyze remote data on Zeus - C-GLORSv7

#### Prerequisite: prepare the environment 

1. Connect to the Zeus VPN using the VPN client, e.g. TunnelBlick

2. Add your local public key in the .ssh/authorized_keys on your Zeus account

3. Create the full conda environment starting from the requirements listed in *environment.yml* by running the following commands:

`conda env create -f environment.yml`

`source activate zapata`

Import Python utilities and setup Zeus username (replace YOURUSERNAME with your account)

In [1]:
import os, sys
sys.path.append("../")
import zeus
zeus.init(user="tl28319",host='zeus')

Start a Dask client on Zeus with 1 worker node (*n_workers = 1*)

In [2]:
from distributed import Client, get_client
client = zeus.start_dask(name="Zapata",
              project="R000",
              cores=36,
              processes=12,
              memory="50 GB",
              interface="ib0",
              walltime="00:30", 
              job_extra=["-x"], 
              n_workers=1
             )

/users_home/oda/tl28319/dask-space/connection
Scheduler address is: tcp://192.168.118.12:42711
Job <17165> is submitted to queue <p_short>.



/Users/lovato/miniconda3/envs/zapata/lib/python3.7/site-packages/distributed/client.py:1078: VersionMismatchWarning: Mismatched versions found

cloudpickle
+-----------+---------+
|           | version |
+-----------+---------+
| client    | 1.6.0   |
| scheduler | 1.3.0   |
+-----------+---------+
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))


Check if it the cluster has been allocated on LSF (some time is required before LSF provides the resources; try to run the cell again after a few seconds). When available click on the **Dashboard** link to open the Dask monitoring interface.

In [5]:
client

Client Scheduler: tcp://192.168.118.12:42711 Dashboard: http://192.168.118.12:39802/status,Cluster Workers: 12 Cores: 36 Memory: 50.00 GB


Let's load the C-GLORSv7 dataset in parallel using Xarray using zapata data interface.

In [6]:
def read_remote():
    import sys
    sys.path.append('/users_home/oda/tl28319/GIT/Zapata')                       
    import zapata.data as zdat
    client = get_client()
    thedata = zdat.read_xarray(dataset='C-GLORSv7', var='votemper', period=[2000, 2001], season='JFM', level=[500.])
    return thedata

future = client.submit(read_remote, "args")
temp = future.result()

AttributeError: module 'types' has no attribute 'CellType'

Show the computation result

In [ ]:
print(temp)

Create a map of the average temperature using zapata mapping tools.

In [ ]:
import zapata.mapping as zmap

Shutdown the cluster and release the resources on LSF. Note sometimes warnings might be raised by the function due to synchronization effects.

In [9]:
zeus.stop_dask(client)

RuntimeError: IOLoop is closed